## Using Azure OpenAI with Azure AI Search with User Assigned Identity

Before we start we need to login using the Azure CLI.

In [ ]:
az login

With that done we need to define some variables for us to continue

In [ ]:
resource_group_name="openai-sample-rg"
openai_resource_name="openai-sample-umi"
openai_deployment_model="gpt-4o"
openai_deployment_name="gpt-4o"
# Capacity in thousands of tokens per minute
openai_deployment_capacity="150"
azure_ai_search_name="openai-sample-search"
azure_ai_search_datasource_name="openai-sample-datasource"
azure_ai_search_index_name="openai-sample-index"
azure_ai_search_indexer_name="openai-sample-indexer"
storage_account_name="openaisamplestorage"
storage_file_share_name="openaisamplefileshare"
identity_name="openai-sample-identity"
location="eastus"


1. Create a Resource Group

We need to create a resource group to put all of the resources we will create

In [ ]:
az group create \
    --name $resource_group_name \
    --location $location

2. Create an Azure OpenAI resource


In [ ]:
az cognitiveservices account create \
    --name $openai_resource_name \
    --resource-group $resource_group_name \
    --custom-domain $openai_resource_name \
    --location $location \
    --kind OpenAI \
    --sku s0

3. Get Azure OpenAI resource endpoint

In [ ]:
azure_openai_endpoint=`az cognitiveservices account show \
    --name $openai_resource_name \
    --resource-group  $resource_group_name \
    | jq -r .properties.endpoint`
echo $azure_openai_endpoint

With that done save the endpoint to your `.env` file in the `AZURE_OPENAI_ENDPOINT` variable.

4. Deploy a model

In [ ]:
az cognitiveservices account deployment create \
    --name $openai_resource_name \
    --resource-group  $resource_group_name \
    --deployment-name $openai_deployment_name \
    --model-name $openai_deployment_model \
    --model-version "2024-05-13" \
    --model-format OpenAI \
    --capacity $openai_deployment_capacity \
    --sku-name "Standard"

Save your deployment name to your `.env` file in the `AZURE_OPENAI_DEPLOYMENT` variable.

5. Assign the `Cognitive Services OpenAI Contributor` role to your account.

First, we find the id for the role.

In [ ]:
openai_contributor_role_id=`az role definition list \
    --query "[?roleName=='Cognitive Services OpenAI Contributor'].{id:id}" \
    | jq -r .[0].id`
echo $openai_contributor_role_id

We also need the id of the Azure OpenAI resource.

In [ ]:
openai_resource_id=`az resource show \
    --name $openai_resource_name \
    --resource-group $resource_group_name \
    --resource-type Microsoft.CognitiveServices/accounts \
    | jq -r .id`
echo $openai_resource_id

Finally, we create the assignment

In [ ]:
user_id=`az ad signed-in-user show \
    | jq -r .id`

az role assignment create \
    --assignee $user_id \
    --role $openai_contributor_role_id \
    --scope $openai_resource_id

6. Create a storage account

In [ ]:
az storage account create \
    --resource-group $resource_group_name \
    --name $storage_account_name \
    --location $location \
    --kind StorageV2 \
    --sku Standard_LRS \
    --enable-large-file-share \
    --output none

Once created, get the connection string for it.

In [ ]:
storage_connection_string=`az storage account show-connection-string \
    --name $storage_account_name \
    --resource-group $resource_group_name \
    | jq -r .connectionString`
echo $storage_connection_string

7. Create an Azure File Share

In [ ]:
az storage share-rm create \
    --resource-group $resource_group_name \
    --storage-account $storage_account_name \
    --name $storage_file_share_name \
    --quota 1024 \
    --enabled-protocols SMB \
    --output none

8. Create a directory and upload the file you want to interact with

In [ ]:
az storage directory create \
   --account-name $storage_account_name \
   --connection-string $storage_connection_string \
   --share-name $storage_file_share_name \
   --name "files" \
   --output none

And then upload the file to the directory we just created.

In [ ]:
az storage file upload \
    --account-name $storage_account_name \
    --connection-string $storage_connection_string \
    --share-name $storage_file_share_name \
    --source "../../../data/Benefit_Options.pdf" \
    --path "files/Benefit_Options.pdf"

9. Create your Azure AI Search resource

In [ ]:
az search service create \
    --name $azure_ai_search_name \
    --resource-group $resource_group_name \
    --sku Standard \
    --partition-count 1 \
    --replica-count 1

Note that this could take a long time.

Once created we save the id and the endpoint of the search resource.

In [ ]:
azure_ai_search_id=`az search service show \
    --name $azure_ai_search_name \
    --resource-group $resource_group_name \
    | jq -r .id`

echo $azure_ai_search_id

azure_ai_search_endpoint="https://$azure_ai_search_name.search.windows.net"
echo $azure_ai_search_endpoint

10. Obtain the Azure AI Search resource admin key

In [ ]:
admin_key=`az search admin-key show \
    --service-name $azure_ai_search_name \
    --resource-group $resource_group_name \
    | jq -r .primaryKey`


11. Create a data source

In [ ]:
request='{
    "name": "'${azure_ai_search_datasource_name}'",
    "type": "azurefile",
    "credentials": {
        "connectionString": "'${storage_connection_string}'"
    },
    "container": {
        "name": "'${storage_file_share_name}'",
        "query" : "files"
    }
}'

curl -X POST \
    -H "Content-Type: application/json" \
    -H "api-key: $admin_key" \
    --data-binary "$request" \
    "$azure_ai_search_endpoint/datasources?api-version=2024-05-01-preview"



12. Create an index

In [ ]:
request='{
  "name" : "'${azure_ai_search_index_name}'",
  "fields": [
      { "name": "ID", "type": "Edm.String", "key": true, "searchable": false },
      { "name": "content", "type": "Edm.String", "searchable": true, "filterable": false },
      { "name": "metadata_storage_name", "type": "Edm.String", "searchable": false, "filterable": true, "sortable": true  },
      { "name": "metadata_storage_path", "type": "Edm.String", "searchable": false, "filterable": true, "sortable": true },
      { "name": "metadata_storage_size", "type": "Edm.Int64", "searchable": false, "filterable": true, "sortable": true  },
      { "name": "metadata_storage_content_type", "type": "Edm.String", "searchable": true, "filterable": true, "sortable": true }
  ]
}'

curl -X POST \
    -H "Content-Type: application/json" \
    -H "api-key: $admin_key" \
    --data-binary "$request" \
    "$azure_ai_search_endpoint/indexes?api-version=2023-11-01"

13. Create the indexer

In [ ]:
request='{
  "name" : "'${azure_ai_search_indexer_name}'",
  "dataSourceName" : "'${azure_ai_search_datasource_name}'",
  "targetIndexName" : "'${azure_ai_search_index_name}'",
  "parameters": {
     "batchSize": null,
     "maxFailedItems": null,
     "maxFailedItemsPerBatch": null,
     "base64EncodeKeys": null,
     "configuration": {
        "indexedFileNameExtensions" : ".pdf,.docx",
        "excludedFileNameExtensions" : ".png,.jpeg"
    }
  },
  "schedule" : { },
  "fieldMappings" : [ ]
}'

curl -X POST \
    -H "Content-Type: application/json" \
    -H "api-key: $admin_key" \
    --data-binary "$request" \
    "$azure_ai_search_endpoint/indexers?api-version=2023-11-01"

14. Create an identity

In [ ]:
az identity create \
    --resource-group $resource_group_name \
    --name $identity_name

Once created get the principal id and the id.

In [ ]:
identity_info=`az identity show \
    --resource-group $resource_group_name \
    --name $identity_name`

identity_principal_id=`echo $identity_info | jq -r .principalId`
echo $identity_principal_id

identity_id=`echo $identity_info | jq -r .id`
echo $identity_id


Save the id to your `.env` file in the variable `MANAGED_IDENTITY_RESOURCE_ID`.

15. Assign the identity to your Azure OpenAI resource

In [ ]:
request='{
    "identity": {
        "type": "UserAssigned",
        "userAssignedIdentities": {
            "'${identity_id}'": {}
        }
    }
}'

az rest \
    --method patch \
    --uri https://management.azure.com${openai_resource_id}?api-version=2023-05-01 \
    --body "$request"


16. Enable EntraId auth in your search resource

In [ ]:
az search service update \
    --name $azure_ai_search_name \
    --resource-group $resource_group_name \
    --auth-options aadOrApiKey \
    --aad-auth-failure-mode http401WithBearerChallenge

17. Assign the `Search Service Contributor` role to your identity.

In [ ]:
role_id=`az role definition list \
    --query "[?roleName=='Search Service Contributor'].{id:id}" \
    | jq -r ".[0].id"`
az role assignment create \
    --assignee $identity_principal_id \
    --role $role_id \
    --scope $azure_ai_search_id